## Import Packages

In [1]:
import jupyterlab_nodeeditor as jlne
from yggdrasil import yamlfile

## Preparation

**set up socket types for node editor**

In [2]:
schema = yamlfile.get_schema()
socket_types = tuple(schema.form_schema['definitions']['schema']['definitions']['simpleTypes']['enum'])
coll = jlne.SocketCollection(socket_types = socket_types)

**load yml file**

In [3]:
filename = "./gs_lesson4/gs_lesson4_python.yml"
model_set = yamlfile.parse_yaml(filename, model_only=True)

In [4]:
# model_set["models"][0].keys()

In [5]:
# model_set["models"][0]["inputs"]

## Transformation

In [6]:
def transform(model_set,coll):
    all_nodes=[]
    node_id=1
    for model in model_set["models"]:
        inputs=[]
        outputs=[]
        
        ## currently we only consider inputs/outputs contains on input/output
        for input__ in model["inputs"]:
            inputs.append(input__["name"].split(":")[1])
        
        for output__ in model["outputs"]:
            outputs.append(output__["name"].split(":")[1])
        
        ## we create the ints and outs below. at the end return the nodes
        input_param=1
        output_parm=1
        input_ls=[]
        output_ls=[]
        for input_ in inputs:
            locals()['int_{}'.format(input_param)] = jlne.InputSlot(title = input_, key='int_{}'.format(input_param) ,sockets = coll,  multi_connection=True)
            input_ls.append(locals()['int_{}'.format(input_param)])
            input_param=input_param+1
            
        for output_ in outputs:
            locals()['out_{}'.format(output_parm)] = jlne.OutputSlot(title = output_, key='out_{}'.format(output_parm),sockets = coll,  multi_connection=True)
            output_ls.append(locals()['out_{}'.format(output_parm)] )
            output_parm=output_parm+1
        
        
        locals()['node_{}'.format(node_id)]=jlne.node_editor.NodeInstanceModel(title="Custom Node {}".format(node_id),inputs=input_ls, outputs=output_ls)
        all_nodes.append(locals()['node_{}'.format(node_id)])
        node_id=node_id+1

    return(all_nodes)

In [7]:
new_nodes=transform(model_set,coll)

## [Editor] Visualize the Components

In [8]:
editor = jlne.NodeEditor(socket_types=socket_types)
editor

In [9]:
for node in new_nodes:
    editor.node_editor.nodes=editor.node_editor.nodes+[node]